<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment01/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

#Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

﻿

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
label = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(10):
   wandb.init(project="cs6910-assignment01")
   wandb.run.name = "Q1_run_{}".format(i+1)
   for j in range(30):
     if y_train[j] == i:
       wandb.log({"examples": [wandb.Image(X_train[j], caption=label[i])]})
       break

# Question 2 (10 Marks)

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [2]:
import numpy as np


def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

def tanh(z):
  g = np.tanh(z)
  return g

def relu(z):
  return np.maximum(0,z)

In [3]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [4]:
def initialize(initializer, size1, size2):
  if(initializer == "random"):
     W = np.random.randn(size1, size2) * 0.01
     return W

  if(initializer == "Xavier"):
     W = np.random.randn(size1, size2) * np.sqrt(1/size2)
     return W

  print("Enter the name of initializer correctly")

In [5]:
def linear_forward(H, W, b):
  W = np.asarray(W)
  H = np.reshape(H,(H.shape[0],-1))
  A = np.dot(W,H) + b 
  cache = (H, W, b)
  
  return A, cache

In [6]:
def initialize_Wb_matrix(X, num_hidden, size_hidden, initializer):
  layer_dims = [X.shape[0]]
  for l in range(0, num_hidden):
    layer_dims.append(size_hidden)
  layer_dims.append(10)   
  np.random.seed(3)
  Wb_matrix = {}
  update = {}
  grads = {}
  L = len(layer_dims)            # number of layers in the network

  for l in range(1,L):
    Wb_matrix['W' + str(l)] = initialize(initializer, layer_dims[l], layer_dims[l-1])
    update['W' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    Wb_matrix['b' + str(l)] = np.zeros((layer_dims[l], 1))
    update['b' + str(l)] = np.zeros((layer_dims[l], 1))
    grads['dW' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    grads['db' + str(l)] = np.zeros((layer_dims[l], 1))
  return Wb_matrix, update, grads

In [7]:
def activation_forward(H_prev, W, b, activation):
  A, linear_cache = linear_forward(H_prev, W, b)
  activation_cache = A
  if activation == 'relu':
    H = relu(A)
  elif activation == 'sigmoid':
    H = sigmoid(A)
  elif activation =='tanh':
    H = tanh(A)
  elif activation == 'softmax':
    H = softmax(A)
  
  return H, activation_cache, linear_cache

In [8]:
def forward_propagation(X, Wb_matrix, activation):
    H = X
    L = int((len(Wb_matrix)/2))
    A_caches = []
    H_caches = [H]
    for l in range(1, L):
        H_prev = H 
        H, A_cache, linear_cache = activation_forward(H_prev, Wb_matrix['W{:d}'.format(l)], Wb_matrix['b{:d}'.format(l)], activation)
        A_caches.append(A_cache)
        H_caches.append(H)
    HL, AL, linear_cache = activation_forward(H, Wb_matrix['W%d' % L], Wb_matrix['b%d' % L], activation='softmax')
    A_caches.append(AL)
    H_caches.append(HL)
    return HL, H_caches, A_caches

# Question 3 (18 Marks)

Implement the backpropagation algorithm with support for the following optimisation functions 

- sgd
- momentum based gradient descent
- nesterov accelerated gradient descent
- rmsprop
- adam
- nadam

(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)

We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.

In [9]:
def one_hot_encoding(l, L):
  import numpy
  e = []
  for i in range(L):
    if i == l:
      e.append(1)
    else:
      e.append(0)
  e_y = np.asarray(e)
  return e_y

In [10]:
def deriv_activ(z, activation):
  if(activation == "relu"):
    z[z<=0] = 0
    z[z>0] = 1
    return z

  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv
  
  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2
    return deriv

In [11]:
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation, weight_decay):
  e_y = []
  L = int((len(Wb_matrix)/2))
  num_train = y_hat.shape[1]
  num_classes = y_hat.shape[0]
  for j in range(len(y)):
    e_y.append(one_hot_encoding(y[j],num_classes))
  e_y = np.reshape(e_y,(num_classes,num_train))
  da = -(e_y - y_hat)
  grads = {}
  
  for k in reversed(range(1,L+1)):
    dW = np.matmul(da, np.transpose(H_caches[k-1])) + 2*weight_decay*Wb_matrix['W{:d}'.format(k)]
    db = da
    if k != 1:
      dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
      da = np.multiply(dh, deriv_activ(A_caches[k-2], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db
  
  return grads

In [12]:
def sgd(Wb_matrix, grads, eta,L):
  for l in range(L):
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix['b' + str(l+1)] = Wb_matrix['b' + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update,L):
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)]) + eta * grads['dW' + str(l+1)]
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]
  
  return Wb_matrix, update

def nag(Wb_matrix,eta,gamma, update, L):
  Wb_matrix_temp= {}
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)])
    Wb_matrix_temp['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)])
    Wb_matrix_temp["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix_temp, update

def rmsprop(Wb_matrix, eta, update ,beta1, eps, grads, L):
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(update['W' + str(l+1)]+eps)) * grads['dW' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(update['b' + str(l+1)]+eps)) * grads['db' + str(l+1)]
  return Wb_matrix, update
  
def adam(Wb_matrix, eta, beta1, beta2, update, v_n, eps, grads, L,i):
  
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]
    update['b' + str(l+1)] = beta1 * update['b' + str(l+1)] + (1-beta1) * grads['db' + str(l+1)]
    
    v_n['W' + str(l+1)] = beta2 * v_n['W' + str(l+1)] + (1-beta2) * grads['dW' + str(l+1)]**2
    v_n["b" + str(l+1)] = beta2 * v_n["b" + str(l+1)] + (1-beta2) * grads['db' + str(l+1)]**2
    
    m_hat['W' + str(l+1)] = update['W' + str(l+1)]/(1-math.pow(beta1,i+1))
    m_hat['b' + str(l+1)] = update['b' + str(l+1)]/(1-math.pow(beta1,i+1))

    v_hat['W' + str(l+1)] = v_n['W' + str(l+1)]/(1-math.pow(beta2,i+1))
    v_hat['b' + str(l+1)] = v_n['b' + str(l+1)]/(1-math.pow(beta2,i+1))


    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/(np.sqrt(v_hat['W' + str(l+1)]+eps)))* m_hat['W' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/(np.sqrt(v_hat['b' + str(l+1)]+eps)))* m_hat['b' + str(l+1)]

 
  return Wb_matrix, update, v_n

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
import numpy as np
import math
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train_flatten = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255.0
X_train_flatten = X_train_flatten.T
num_train = X_train.shape[0]


max_epoch = 5
learning_rate = 0.01
batch_size = 64
num_hidden = 4
L = num_hidden+1
size_hidden = 128
weight_decay = 0.0005
initializer = 'Xavier'
activation = 'sigmoid'
optimizer = 'nadam'
beta1 = 0.9
beta2 = 0.999
eps = 1e-8
gamma = 0.9


Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, num_hidden , size_hidden ,initializer)
v = update.copy()
m_hat = update.copy()
v_hat = update.copy()

 
for i in range(max_epoch):
  loss = 0;
  count = 0;
  grads = grad_initial
  print("Epoch:", i+1)
  for j in range(num_train):
    B = np.reshape(X_train_flatten[:,j], (-1, 1))   
    y_hat, H_caches, A_caches = forward_propagation(B, Wb_matrix ,activation)
    if(y_train[j]==np.argmax(y_hat)):
      count = count + 1; 

    if (j+1)%batch_size == 0  :
      if optimizer == 'nag':
        Wb_matrix, update = nag(Wb_matrix, learning_rate,gamma, update, L)
      if optimizer == 'nadam':
        Wb_matrix, update = nag(Wb_matrix, learning_rate,gamma, update, L)

    grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], activation, weight_decay)
    for l in range(L):
      grads['dW' + str(l+1)] = grads['dW' + str(l+1)] + grad['dW' + str(l+1)] 
      grads['db' + str(l+1)] = grads['db' + str(l+1)] + grad['db' + str(l+1)] 


    if (j+1)%batch_size == 0 : 
      for l in range(L):
        grads['dW' + str(l+1)] = grads['dW' + str(l+1)]/batch_size
        grads['db' + str(l+1)] = grads['db' + str(l+1)]/batch_size
      if optimizer == 'sgd':
        Wb_matrix = sgd(Wb_matrix, grads, learning_rate,L)
      elif optimizer == 'momentum':
        Wb_matrix,update = momentum(Wb_matrix,grads,learning_rate,gamma,update,L)
      elif optimizer == 'nag':
        Wb_matrix,update = momentum(Wb_matrix,grads,learning_rate,gamma,update,L)
      elif optimizer == 'rmsprop':
        Wb_matrix, update = rmsprop(Wb_matrix, learning_rate, update,  beta1, eps, grads, L)
      elif optimizer == 'adam':
        Wb_matrix, update,v = adam(Wb_matrix, learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
      elif optimizer == 'nadam':
        Wb_matrix, update,v = adam(Wb_matrix, learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
    
    loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
  print("     Loss", loss)  
  acc = 100* count/num_train
  print("     Accuracy",acc)

Epoch: 1
